In [5]:
import pandas as pd
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import requests
import json
from pprint import pprint


### Extract CSVs into DataFrames

In [21]:

sales_df = "sales.csv"
sales_df = pd.read_csv(sales_df, encoding='Latin-1')
sales_df.head(50)

sales_df.columns= sales_df.columns.str.lower()
for col in sales_df.columns:
    print(col)
    
sales_df.head()


ordernumber
quantityordered
priceeach
orderlinenumber
sales
orderdate
status
qtr_id
month_id
year_id
productline
msrp
productcode
customername
phone
addressline1
addressline2
city
state
postalcode
country
territory
contactlastname
contactfirstname
dealsize


,ordernumber,quantityordered,priceeach,orderlinenumber,sales,orderdate,status,qtr_id,month_id,year_id,...,addressline1,addressline2,city,state,postalcode,country,territory,contactlastname,contactfirstname,dealsize
0,10107,30,95.70,2,2871.00,2/24/2003 0:00,Shipped,1,2,2003,...,897 Long Airport Avenue,NaN,NYC,NY,10022,USA,NaN,Yu,Kwai,Small
1,10121,34,81.35,5,2765.90,5/7/2003 0:00,Shipped,2,5,2003,...,59 rue de l'Abbaye,NaN,Reims,NaN,51100,France,EMEA,Henriot,Paul,Small
2,10134,41,94.74,2,3884.34,7/1/2003 0:00,Shipped,3,7,2003,...,27 rue du Colonel Pierre Avia,NaN,Paris,NaN,75508,France,EMEA,Da Cunha,Daniel,Medium
3,10145,45,83.26,6,3746.70,8/25/2003 0:00,Shipped,3,8,2003,...,78934 Hillside Dr.,NaN,Pasadena,CA,90003,USA,NaN,Young,Julie,Medium
4,10159,49,100.00,14,5205.27,10/10/2003 0:00,Shipped,4,10,2003,...,7734 Strong St.,NaN,San Francisco,CA,NaN,USA,NaN,Brown,Julie,Medium


In [22]:
#Transforming the data to Postgres 
rds_connection_string = "postgres:postgres@localhost:5432/Sales_db"
engine = create_engine(f'postgresql://{rds_connection_string}')
engine.table_names()

/var/folders/w0/dyll3qjj14n0d4qd0jv9g_4w0000gn/T/ipykernel_22003/1513601241.py:4: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['sales', 'sales_info']

In [24]:
#connect table to postgres
sales_df.to_sql(name='sales_info1', con=engine, if_exists='append', index=False)
engine.table_names()

/var/folders/w0/dyll3qjj14n0d4qd0jv9g_4w0000gn/T/ipykernel_22003/2058310832.py:3: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['sales', 'sales_info', 'sales_info1']

In [25]:
#testing the table after connecting to ensure connection worked
pd.read_sql_query('select * from sales_info1', con=engine).head()

,ordernumber,quantityordered,priceeach,orderlinenumber,sales,orderdate,status,qtr_id,month_id,year_id,...,addressline1,addressline2,city,state,postalcode,country,territory,contactlastname,contactfirstname,dealsize
0,10107,30,95.70,2,2871.00,2/24/2003 0:00,Shipped,1,2,2003,...,897 Long Airport Avenue,None,NYC,NY,10022,USA,None,Yu,Kwai,Small
1,10121,34,81.35,5,2765.90,5/7/2003 0:00,Shipped,2,5,2003,...,59 rue de l'Abbaye,None,Reims,None,51100,France,EMEA,Henriot,Paul,Small
2,10134,41,94.74,2,3884.34,7/1/2003 0:00,Shipped,3,7,2003,...,27 rue du Colonel Pierre Avia,None,Paris,None,75508,France,EMEA,Da Cunha,Daniel,Medium
3,10145,45,83.26,6,3746.70,8/25/2003 0:00,Shipped,3,8,2003,...,78934 Hillside Dr.,None,Pasadena,CA,90003,USA,None,Young,Julie,Medium
4,10159,49,100.00,14,5205.27,10/10/2003 0:00,Shipped,4,10,2003,...,7734 Strong St.,None,San Francisco,CA,None,USA,None,Brown,Julie,Medium
